## leinforcement pyform


In [1]:
#!/usr/bin/python
import numpy as np
import pandas as pd
import time
from collections import deque
from tqdm import tqdm

from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
from gym import spaces
from gym.spaces.box import Box

import os
import subprocess
import PyFoam
import PyFoam.FoamInformation
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
from PyFoam.Basics.DataStructures import Vector
from PyFoam.Execution.BasicRunner import BasicRunner
from PyFoam.Basics.TemplateFile import TemplateFile
import shlex,sys,json
import re
from pathlib import Path

In [2]:
#PyFOAM基本コマンド 

In [3]:
# set directory
CASE = SolutionDirectory("../aircond5")

In [3]:
subprocess.call("./Allclean")

0

In [4]:
subprocess.call('./Makemesh')

0

In [5]:
# OpenFOAMのコマンドを実行
args=shlex.split("buoyantBoussinesqPimpleFoam -case " + CASE.name)
buoyant=BasicRunner(args,silent=True)
summary=buoyant.start()
buoyant.runOK()

False

# 以下、棒倒しのDQN

In [4]:
# パッケージのimport
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym


In [5]:
# 動画の描画関数の宣言
# 参考URL http://nbviewer.jupyter.org/github/patrickmineault
# /xcorr-notebooks/blob/master/Render%20OpenAI%20gym%20as%20GIF.ipynb
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0),
               dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames),
                                   interval=50)

    anim.save('movie_cartpole_DQN.mp4')  # 動画のファイル名と保存です
    display(display_animation(anim, default_mode='loop'))
    

In [6]:
# 本コードでは、namedtupleを使用します。
# namedtupleを使うことで、値をフィールド名とペアで格納できます。
# すると値に対して、フィールド名でアクセスできて便利です。
# https://docs.python.jp/3/library/collections.html#collections.namedtuple
# 以下は使用例です

from collections import namedtuple

Tr = namedtuple('tr', ('name_a', 'value_b'))
Tr_object = Tr('名前Aです', 100)

print(Tr_object)  # 出力：tr(name_a='名前Aです', value_b=100)
print(Tr_object.value_b)  # 出力：100


tr(name_a='名前Aです', value_b=100)
100


In [7]:
# namedtupleを生成
from collections import namedtuple

Transition = namedtuple(
    'Transition', ('state', 'action', 'next_state', 'reward'))


In [8]:
# 定数の設定
ENV = 'CartPole-v0'  # 使用する課題名
GAMMA = 0.99  # 時間割引率
MAX_STEPS = 200  # 1試行のstep数
NUM_EPISODES = 500  # 最大試行回数


In [9]:
# 経験を保存するメモリクラスを定義します


class ReplayMemory:

    def __init__(self, CAPACITY):
        self.capacity = CAPACITY  # メモリの最大長さ
        self.memory = []  # 経験を保存する変数
        self.index = 0  # 保存するindexを示す変数

    def push(self, state, action, state_next, reward):
        '''transition = (state, action, state_next, reward)をメモリに保存する'''

        if len(self.memory) < self.capacity:
            self.memory.append(None)  # メモリが満タンでないときは足す

        # namedtupleのTransitionを使用し、値とフィールド名をペアにして保存します
        self.memory[self.index] = Transition(state, action, state_next, reward)

        self.index = (self.index + 1) % self.capacity  # 保存するindexを1つずらす

    def sample(self, batch_size):
        '''batch_size分だけ、ランダムに保存内容を取り出す'''
        return random.sample(self.memory, batch_size)

    def __len__(self):
        '''関数lenに対して、現在の変数memoryの長さを返す'''
        return len(self.memory)


In [10]:
# エージェントが持つ脳となるクラスです、DQNを実行します
# Q関数をディープラーニングのネットワークをクラスとして定義

import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000


class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  # CartPoleの行動（右に左に押す）の2を取得

        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワークを構築
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))

        print(self.model)  # ネットワークの形を出力

        # 最適化手法の設定
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)

    def replay(self):
        '''Experience Replayでネットワークの結合パラメータを学習'''

        # -----------------------------------------
        # 1. メモリサイズの確認
        # -----------------------------------------
        # 1.1 メモリサイズがミニバッチより小さい間は何もしない
        if len(self.memory) < BATCH_SIZE:
            return

        # -----------------------------------------
        # 2. ミニバッチの作成
        # -----------------------------------------
        # 2.1 メモリからミニバッチ分のデータを取り出す
        transitions = self.memory.sample(BATCH_SIZE)

        # 2.2 各変数をミニバッチに対応する形に変形
        # transitionsは1stepごとの(state, action, state_next, reward)が、BATCH_SIZE分格納されている
        # つまり、(state, action, state_next, reward)×BATCH_SIZE
        # これをミニバッチにしたい。つまり
        # (state×BATCH_SIZE, action×BATCH_SIZE, state_next×BATCH_SIZE, reward×BATCH_SIZE)にする
        batch = Transition(*zip(*transitions))

        # 2.3 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
        # 例えばstateの場合、[torch.FloatTensor of size 1x4]がBATCH_SIZE分並んでいるのですが、
        # それを torch.FloatTensor of size BATCH_SIZEx4 に変換します
        # 状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
        # catはConcatenates（結合）のことです。
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])

        # -----------------------------------------
        # 3. 教師信号となるQ(s_t, a_t)値を求める
        # -----------------------------------------
        # 3.1 ネットワークを推論モードに切り替える
        self.model.eval()

        # 3.2 ネットワークが出力したQ(s_t, a_t)を求める
        # self.model(state_batch)は、右左の両方のQ値を出力しており
        # [torch.FloatTensor of size BATCH_SIZEx2]になっている。
        # ここから実行したアクションa_tに対応するQ値を求めるため、action_batchで行った行動a_tが右か左かのindexを求め
        # それに対応するQ値をgatherでひっぱり出す。
        state_action_values = self.model(state_batch).gather(1, action_batch)

        # 3.3 max{Q(s_t+1, a)}値を求める。ただし次の状態があるかに注意。

        # cartpoleがdoneになっておらず、next_stateがあるかをチェックするインデックスマスクを作成
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None,
                                                    batch.next_state)))
        # まずは全部0にしておく
        next_state_values = torch.zeros(BATCH_SIZE)

        # 次の状態があるindexの最大Q値を求める
        # 出力にアクセスし、max(1)で列方向の最大値の[値、index]を求めます
        # そしてそのQ値（index=0）を出力します
        # detachでその値を取り出します
        next_state_values[non_final_mask] = self.model(
            non_final_next_states).max(1)[0].detach()

        # 3.4 教師となるQ(s_t, a_t)値を、Q学習の式から求める
        expected_state_action_values = reward_batch + GAMMA * next_state_values

        # -----------------------------------------
        # 4. 結合パラメータの更新
        # -----------------------------------------
        # 4.1 ネットワークを訓練モードに切り替える
        self.model.train()

        # 4.2 損失関数を計算する（smooth_l1_lossはHuberloss）
        # expected_state_action_valuesは
        # sizeが[minbatch]になっているので、unsqueezeで[minibatch x 1]へ
        loss = F.smooth_l1_loss(state_action_values,
                                expected_state_action_values.unsqueeze(1))

        # 4.3 結合パラメータを更新する
        self.optimizer.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算
        self.optimizer.step()  # 結合パラメータを更新

    def decide_action(self, state, episode):
        '''現在の状態に応じて、行動を決定する'''
        # ε-greedy法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # ネットワークを推論モードに切り替える
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)
            # ネットワークの出力の最大値のindexを取り出します = max(1)[1]
            # .view(1,1)は[torch.LongTensor of size 1]　を size 1x1 に変換します

        else:
            # 0,1の行動をランダムに返す
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])  # 0,1の行動をランダムに返す
            # actionは[torch.LongTensor of size 1x1]の形になります

        return action


In [11]:
# CartPoleで動くエージェントクラスです、棒付き台車そのものになります


class Agent:
    def __init__(self, num_states, num_actions):
        '''課題の状態と行動の数を設定する'''
        self.brain = Brain(num_states, num_actions)  # エージェントが行動を決定するための頭脳を生成

    def update_q_function(self):
        '''Q関数を更新する'''
        self.brain.replay()

    def get_action(self, state, episode):
        '''行動を決定する'''
        action = self.brain.decide_action(state, episode)
        return action

    def memorize(self, state, action, state_next, reward):
        '''memoryオブジェクトに、state, action, state_next, rewardの内容を保存する'''
        self.brain.memory.push(state, action, state_next, reward)


In [12]:
class Test:
    def __init__(self,a,b):
        self.a = a
        self.b = b
        
    def a_plus_b(self):
        return self.a+self.b
    def aaa(self):
        self.c = self.a + self.b

In [13]:
# PyFoamで最適条件を探す用

class pyfoamtest:
    
    def __init__(self,CASE):
        
        self.CASE = CASE
        
        # get nCells
        with open (self.CASE.name + '/constant/polyMesh/neighbour') as f:
            neighbour = f.read()
        nCells_index = neighbour.find('nCells')
        nCells_ = neighbour[nCells_index : nCells_index+15]
        nCells = int(re.sub(r'\D', '', nCells_))
        self.nCells = nCells
        
        # 各辞書ファイルの取得
        self.initialDir = self.CASE.initialDir()+'/'
        self.constant = self.CASE.name + "/constant/"
        self.system = self.CASE.name + "/system/"
        self.initialDir_file = []
        for x in os.listdir(self.initialDir):
            if os.path.isfile(self.initialDir + x):
                self.initialDir_file.append(x)
        self.constant_file = []
        for y in os.listdir(self.constant):
            if os.path.isfile(self.constant + y):
                self.constant_file.append(y)
        self.system_file = []
        for z in os.listdir(self.system):
            if os.path.isfile(self.system + z):
                self.system_file.append(z)
        
        # 各辞書ファイルをそれぞれのファイル名で保存
        for i in range(len(self.initialDir_file)):
            self.__dict__[self.initialDir_file[i]] = ParsedParameterFile(self.initialDir + self.initialDir_file[i])
        for i in range(len(self.system_file)):
            self.__dict__[self.system_file[i]] = ParsedParameterFile(self.system + self.system_file[i])

In [285]:
## この環境の部分を変える

class Aircond:
    '''Aircondのクラス'''
    def __init__(self,CASE):
        self.CASE = CASE
        
        # get nCells
        with open (self.CASE.name + '/constant/polyMesh/neighbour') as f:
            neighbour = f.read()
        nCells_index = neighbour.find('nCells')
        nCells_ = neighbour[nCells_index : nCells_index+15]
        nCells = int(re.sub(r'\D', '', nCells_))
        self.nCells = nCells
        
        self.action_SPEED = np.array([0.1,0.3,0.5])
        self.action_DIRECTION = np.array([-1*np.pi/8, -2*np.pi/8,-3*np.pi/8])
        self.action_TEMPERTURE = np.array([18,22,26])
        self.action_space = np.tile(np.array([0,0,0]),(27,1))
        self.observation_space = np.tile(np.array([0,0,0]),(self.nCells,1))
        self.stride = 1200
        self.startTime = 0  # startTimeの初期化
        self.endTime = 1200  # 初期化
        self.end = 20000
        
        
        # 各辞書ファイルの取得
        self.initialDir = self.CASE.initialDir()+'/'
        self.constant = self.CASE.name + "/constant/"
        self.system = self.CASE.name + "/system/"
        self.initialDir_file = []
        for x in os.listdir(self.initialDir):
            if os.path.isfile(self.initialDir + x):
                self.initialDir_file.append(x)
        self.constant_file = []
        for y in os.listdir(self.constant):
            if os.path.isfile(self.constant + y):
                self.constant_file.append(y)
        self.system_file = []
        for z in os.listdir(self.system):
            if os.path.isfile(self.system + z):
                self.system_file.append(z)
        
        # 各辞書ファイルをそれぞれのファイル名で保存
        for i in range(len(self.initialDir_file)):
            self.__dict__[self.initialDir_file[i]] = ParsedParameterFile(self.initialDir + self.initialDir_file[i])

        for i in range(len(self.system_file)):
            self.__dict__[self.system_file[i]] = ParsedParameterFile(self.system + self.system_file[i])
            
    def initial_to_float(self, numpy_Parsed_value):
        '''uniformをnp.arrayに変換'''
        if numpy_Parsed_value.ndim==0:
            Parsed_raw = str(numpy_Parsed_value.all())
            Parsed_str = Parsed_raw[8:].strip('()').split(' ')
            Parsed_int = np.array(list(map(float,Parsed_str)))
            #Parsed = np.tile(Parsed_int,(self.nCells,1))
        return Parsed_int
    
    def initial_to_array(self, numpy_Parsed_value):
        '''uniformをnCellの数だけnp.arrayに変換'''
        if numpy_Parsed_value.ndim==0:
            Parsed_raw = str(numpy_Parsed_value.all())
            Parsed_str = Parsed_raw[8:].strip('()').split(' ')
            Parsed_int = np.array(list(map(float,Parsed_str)))
            Parsed = np.tile(Parsed_int,(self.nCells,1))
        return Parsed

    def make_observation(self,Dir):
        '''Dirのpathのobservationを取得'''
        U_value = np.array(ParsedParameterFile(Dir + '/U').content['internalField'])
        T_value = np.array(ParsedParameterFile(Dir + '/T').content['internalField'])
        if U_value.ndim == 0:
            U_value = self.initial_to_array(U_value)
            T_value = self.initial_to_array(T_value)
        U_value_xy = np.delete(U_value, axis=1, obj=2)
        T_value_x = np.reshape(T_value, [-1,1], order='F')
        Observation = np.concatenate([U_value_xy, T_value_x],axis=1)
        return Observation    
    
    def make_action(self):
        '''actionの設定'''
        Action = np.empty((0,3),float)
        for i in range(len(self.action_SPEED)):
            for j in range(len(self.action_DIRECTION)):
                for k in range(len(self.action_TEMPERTURE)):
                    Ux = self.action_SPEED[i]*np.cos(self.action_DIRECTION[j])
                    Uy = self.action_SPEED[i]*np.sin(self.action_DIRECTION[j])
                    Act = np.array([[Ux,Uy,self.action_TEMPERTURE[k]]])
                    Action = np.append(Action,Act,axis=0)
                    
        return Action
    
    def getParsed(self,time_step):
        '''各time_stepのParsedParameterFileを取得'''
        T = ParsedParameterFile(CASE.name + '/' + str(time_step) + '/T')
        U = ParsedParameterFile(CASE.name + '/' + str(time_step) + '/U')
        TU_list = [T,U]
        return TU_list
    
    
    def getParsedList(self,first_step, last_step, write_step,):
        '''各time_stepのParsedParameterFileを取得'''
        TU_list = []
        for stp in range(first_step, last_step, write_step):
            T = ParsedParameterFile(CASE.name + '/' + str(stp) + '/T')
            U = ParsedParameterFile(CASE.name + '/' + str(stp) + '/U')
            TU_list.append([T,U])
        return TU_list
    
    # 後にcythonで書き直す予定
    def calc_PMV(self, TA=20,VA=0.3,TR=20,RH=50,AL=1,CLO=1):
        '''PMVとPPDを計算'''
        #AL = 1  # 活動量[met]
        #CLO = 1 # 着衣量[clo]
        #TA = 20  #  温度[℃]
        #TR = 20  # MRT[℃]
        #VA = 0.3  # 流速[m/s]
        #RH = 50  # 相対湿度[%]
        #
        #***************************************************
        # 外部仕事 W＝0 [W/㎡]とする。
        #***************************************************
        # PMV 計算準備
        #
        M = AL * 58.15
        LCL = CLO
        W = 0
        #PA = (RH / 100 * np.exp(18.6686 - 4030.18 / (TA + 235))) / 0.00750062
        PPK = 673.4 - 1.8 * TA
        PPA = 3.2437814 + 0.00326014 * PPK + 2.00658 * 1E-9 * PPK * PPK * PPK
        PPB = (1165.09 - PPK) * (1 + 0.00121547 * PPK)
        PA = RH / 100 * 22105.8416 / np.exp(2.302585 * PPK * PPA / PPB) * 1000
        EPS = 1E-5
        MW = M - W
        # FCL＝着衣表面積／裸体表面積の比
        if LCL > 0.5:
            FCL = 1.05 + 0.1 * LCL
        else:
            FCL = 1 + 0.2 * LCL
        # 衣服表面温度TCLの初期値設定
        TCL = TA
        TCLA = TCL
        NOI = 1
        # 着衣表面温度の計算
        while True:
            TCLA = 0.8 * TCLA + 0.2 * TCL
            HC = 12.1 * np.sqrt(VA)
            if 2.38 * np.sqrt(np.sqrt(abs(TCL - TA))) > HC:
                HC = 2.38 * np.sqrt(np.sqrt(abs(TCL - TA)))
            TCL = 35.7 - 0.028 * MW - 0.155 * LCL * (3.96 * 1E-8 * FCL * ((TCLA + 273) ** 4 - (TR + 273) ** 4) + FCL * HC * (TCLA - TA))
            NOI = NOI + 1
            if NOI > 150:
                PMV = 999990.999
                PPD = 100
                return (PMV,PPD)
            if not abs(TCLA - TCL) > EPS:
                break
        #PMVの計算
        PM1 = 3.96 * 1E-8 * FCL * ((TCL + 273) ** 4 - (TA + 273) ** 4)
        PM2 = FCL * HC * (TCL - TA)
        PM3 = 0.303 * np.exp(-0.036 * M) + 0.028
        if MW > 58.15:
            PM4 = 0.42 * (MW - 58.15)
        else:
            PM4 = 0
        PMV = PM3 * (MW - 3.05 * 0.001 * (5733 - 6.99 * MW - PA) - PM4 - 1.7 * 1E-5 * M * (5867 - PA) - 0.0014 * M * (34 - TA) - PM1 - PM2)
            #PRINT PMV
        if abs(PMV) > 3:
            PMV = 999990.999
            PPD = 100
            return (PMV,PPD)
        
        PPD = 100 - 95 * np.exp(-0.0335 * PMV ** 4 - 0.2179 * PMV ** 2)
        
        return (PMV,PPD)
    
    def calc_MRT(self, T_Parsed):
        '''MRTを計算'''
        
        T_wall_list = np.array([])
        if np.array(T_Parsed['internalField']).ndim==0:  # time_step=0
            for boundary in list(T_Parsed['boundaryField']):
                if T_Parsed['boundaryField'][boundary]['type']=='zeroGradient' or \
                T_Parsed['boundaryField'][boundary]['type']=='empty' or \
                    T_Parsed['boundaryField'][boundary]['type']=='fixedValue':
                    T_wall = np.array([])
                else:
                    numpy_Parsed_value = np.array(T_Parsed['boundaryField'][boundary]['value'])
                    T_wall = self.initial_to_float(numpy_Parsed_value)
                T_wall_list = np.append(T_wall_list, T_wall)
                
        else:
            for boundary in list(T_Parsed['boundaryField']):
                if T_Parsed['boundaryField'][boundary]['type']=='fixedValue':
                    numpy_Parsed_value = np.array(T_Parsed['boundaryField'][boundary]['value'])
                    T_wall = self.initial_to_float(numpy_Parsed_value)
                elif T_Parsed['boundaryField'][boundary]['type']=='zeroGradient' or \
                T_Parsed['boundaryField'][boundary]['type']=='empty':
                    T_wall = np.array([])
                else:
                    T_wall = np.array(T_Parsed['boundaryField'][boundary]['value'])
                T_wall_list = np.append(T_wall_list, T_wall)
        return np.average(T_wall_list)
    
    def Celsius(self, T):
        CelsiusT = T - 273.15
        return CelsiusT
    
    def Celsius_(self, T):
        '''セルシウス℃に変換'''
        if np.array(T).size==1:
            return self.Celsius(T)
        else:
            Celsiuss = np.frompyfunc(self.Celsius,1,1)  # リストに適用可にする
            return Celsiuss(T)
        
    def UScalar(self, U):
        '''Uをスカラーに変換'''
        if np.array(U).size<=3:
            return [np.sqrt(U[0]**2 + U[1]**2)]
        else:
            return list(np.sqrt(U[:,0]**2 + U[:,1]**2))
        
    def calc_PMV_all(self, TU_Parsed):
        '''PMVを一つのtime_stepで全点計算'''
        
        T_Parsed,U_Parsed = TU_Parsed
        T = np.array(T_Parsed['internalField'])
        U = np.array(U_Parsed['internalField'])
        # time_step==0の場合
        if T.ndim==0 or U.ndim==0:
            T = self.initial_to_float(T)
            U = self.initial_to_float(U)
            # Uを速さに変換
        Us = self.UScalar(U)
        MRT = self.calc_MRT(T_Parsed)
        # TとMRTをセルシウス温度に変換
        Tc = list(self.Celsius_(T))
        MRTc = self.Celsius_(MRT)

        length = len(T)
        # ループを早くするため、外に出す。
        PMV = []
        PPD = []
        PMVappend = PMV.append
        PPDappend = PPD.append
        for i in range(length):
            pmv,ppd = self.calc_PMV(TA=Tc[i],VA=Us[i],TR=MRTc,RH=50,AL=1,CLO=1)
            PMVappend(pmv)
            PPDappend(ppd)
        return [PMV,PPD]
    
    def header(self, time_step, filename):
        '''headerファイルを作成'''
        header = """/*--------------------------------*- C++ -*----------------------------------*\
=========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     | Website:  https://openfoam.org
    \\  /    A nd           | Version:  6
     \\/     M anipulation  |
\*---------------------------------------------------------------------------*/
FoamFile
{{
    version     2.0;
    format      ascii;
    class       volScalarField;
    location    "{}";
    object      {};
}}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //
""".format(time_step, filename)
        return header
    
    def internal(self, list_internal):
        '''internalFieldの値の作成'''
        if len(list_internal)==1:
            internal = """
internalField   uniform {};""".format(list_internal[0])
        else:
            str_= np.frompyfunc(str,1,1)
            str_internal = '\n'.join(str_(list_internal))
            internal = """
internalField   nonuniform List<scalar> 
{}
(
{}
)
;
""".format(self.nCells, str_internal)
        return internal
    
    def makePMVFile(self,time_step):
        '''PMVとPPDファイルを書き込む'''
        
        path_pmv = './{}/PMV'.format(time_step)  # 書き込むパス
        path_ppd = './{}/PPD'.format(time_step)
        
        demensions = """
dimensions      [0 0 0 0 0 0 0];
"""
        
        boundary = """
boundaryField
{
    ".*"
    {
        type            zeroGradient;
    }
}


// ************************************************************************* //
"""
        # header, dimensions, internal, boundaryの順に書き込む
        f = open(path_pmv, 'w') # ファイルを開く(該当ファイルがなければ新規作成)
        g = open(path_ppd, 'w')
        f.write(self.header(time_step,"PMV")) # headerを記載する
        g.write(self.header(time_step,"PPD"))
        f.write(demensions) # dimensionsを記載する
        g.write(demensions)
        # internalFieldの計算
        TU_Parsed = self.getParsed(time_step)
        PMV,PPD = self.calc_PMV_all(TU_Parsed)
        internal_PMV = self.internal(PMV)
        internal_PPD = self.internal(PPD)
        f.write(internal_PMV)  
        g.write(internal_PPD)
        f.write(boundary)
        g.write(boundary)
        f.close() 
        g.close()
        
    def makePMVList(self,first_step, last_step, write_step):
        '''任意の範囲でPMVファイルを作成'''
        for stp in range(first_step, last_step, write_step):
            self.makePMVFile(stp)
            
    def calc_ADPI(self,TU_Parsed):
        '''ADPIを計算する'''
        
        T_Parsed,U_Parsed = TU_Parsed
        T = np.array(T_Parsed['internalField'])
        U = np.array(U_Parsed['internalField'])
        # time_step==0の場合
        if T.ndim==0 or U.ndim==0:
            T = self.initial_to_float(T)
            U = self.initial_to_float(U)
        Tc = np.average(T)  # 室内の平均温度
        Us = self.UScalar(U)  # 流速
        theta = (T - Tc) - 8.0*(Us - 0.15)  # 有効ドラフト温度
        

    def change_control(self,control):
        if control == 1:
            self.blockMeshDict['blocks'][2] = Vector(20,10,1)
            self.blockMeshDict.writeFile()
            self.controlDict['deltaT'] = 0.02
        if control == 2:
            self.blockMeshDict['blocks'][2] = Vector(40,20,1)
            self.blockMeshDict.writeFile()
            self.controlDict['deltaT'] = 0.02
        if control == 3:
            self.blockMeshDict['blocks'][2] = Vector(20,10,1)
            self.blockMeshDict.writeFile()
            self.controlDict['deltaT'] = 0.01
        if control == 4:
            self.blockMeshDict['blocks'][2] = Vector(40,20,1)
            self.blockMeshDict.writeFile()
            self.controlDict['deltaT'] = 0.01
        
    def reset(self):
        '''環境のリセット'''
        
        # reset control Dict
        clDict = ParsedParameterFile(self.CASE.controlDict())
        clDict['startTime'] = 0
        clDict['endTime'] = self.stride
        clDict['deltaT'] = 1
        #clDict['writeInterval'] = 400
        #clDict.writeFile()
        self.startTime = clDict['startTime']
        self.endTime = clDict['endTime']
        
        os.system('./Allclean')
        os.system('./Makemesh')
        
        # 初期条件の設定（ランダム）
        T_initial = ParsedParameterFile(self.CASE.initialDir() + '/T')
        #  random parameter from 26 to 35
        T_rand = np.random.randint(26+273,35+273)
        T_initial['internalField'].setUniform(T_rand)
        T_initial.writeFile()
        
        
        # set action and observation
        self.action_space= self.make_action()
        self.observation = self.make_observation(self.CASE.initialDir())
        return self.observation
    
    def step(self, action):
        '''ステップを進める'''
        
        clDict = ParsedParameterFile(self.CASE.controlDict())      
        if clDict['endTime'] == self.end:
            done = True
        else:
            done = False
            
            # actionに従った、境界条件を設定
            # action is 0~26
            U_latest = ParsedParameterFile(self.CASE.latestDir() + '/U')
            T_latest = ParsedParameterFile(self.CASE.latestDir() + '/T')
            self.act = self.action_space[action]
            U_latest['boundaryField']['inlet']['value'].setUniform(Vector(self.act[0],self.act[1],0))
            U_latest.writeFile()
            T_latest['boundaryField']['inlet']['value'].setUniform(self.act[2])
            T_latest.writeFile()
            
            # OpenFOAMのコマンドを実行
            args=shlex.split("buoyantBoussinesqPimpleFoam -case " + self.CASE.name)
            buoyant=BasicRunner(args,silent=True)
            self.summary=buoyant.start()
            runOK = buoyant.runOK()
            
            #os.system("buoyantBoussinesqPimpleFoam")
            
            # clDictのコントロール
            #clDict = ParsedParameterFile(self.CASE.controlDict())
            #clDict['startTime'] = self.startTime + self.stride
            #clDict['endTime'] = self.endTime + self.stride
            #clDict['deltaT'] = 1
            #clDict['writeInterval'] = 400
            #clDict.writeFile()
            
            self.startTime = clDict['startTime']
            self.endTime = clDict['endTime']
            
            self.observation = self.make_observation(self.CASE.latestDir())
            
        return (self.observation, done, runOK)
        

In [286]:
a = Aircond(CASE)

In [287]:
a.makePMVList(0,200,100)

In [221]:
T_Parsed,U_Parsed = test[0]

In [288]:
T = a.initial_to_float(np.array(T_Parsed['internalField']))

In [222]:
U = a.initial_to_float(np.array(U_Parsed['internalField']))

In [216]:
[np.sqrt(U[0]**2 + U[1]**2)]

[0.0]

In [227]:
PMV,PPD = a.calc_PMV_all(test[0])

In [238]:
len(PMV)

1

In [179]:
T0 = test[0][0]

In [180]:
T1 = test[1][0]

In [159]:
np.array(T0['internalField']).ndim

0

In [157]:
T1['internalField']==float

False

In [141]:
sample = np.array(T0['boundaryField']['floor']['value'])

In [145]:
a.initial_to_float(sample)

array([306.])

In [173]:
a.calc_MRT(T1)

303.3700703125

In [126]:
np.array(test[0][0]['internalField']).ndim

0

In [88]:
PMV,PPD = a.calc_PMV_all(test[1])

In [125]:
T_test,U_test = test[0]

In [35]:
def calc_PMV_all(TU_Parsed):
    T_Parsed,U_Parsed = TU_Parsed
    T = np.array(T_Parsed['internalField'])
    U = np.array(U_Parsed['internalField'])
    MRT = calc_MRT(T_Parsed)
    # TとMRTをセルシウス温度に変換
    def Celsius_(T):
        CelsiusT = T-273.15
        return CelsiusT
    Celsius = np.frompyfunc(Celsius_,1,1) # リストに適用可にする
    Tc = list(Celsius(T))
    MRTc = Celsius_(MRT)
    # Uを速さに変換
    Ua = list(np.sqrt(U[:,0]**2 + U[:,1]**2))
    
    length = len(T)
    # ループを早くするため、外に出す。
    PMV = []
    PPD = []
    PMVappend = PMV.append
    PPDappend = PPD.append
    for i in range(length):
        pmv,ppd = calc_PMV(TA=Tc[i],VA=Ua[i],TR=MRTc,RH=50,AL=1,CLO=1)
        PMVappend(pmv)
        PPDappend(ppd)
    return [PMV,PPD]

In [72]:
P[0]

[0.6180338641846064,
 0.3264824877239651,
 0.548682542796134,
 0.29501308532441783,
 0.4146860037250739,
 0.546941242028897,
 0.3306797251840607,
 0.47175703876910774,
 0.4820305260001097,
 0.6146573378595112,
 0.3887718370724176,
 0.47824370934352434,
 0.7058237629714709,
 0.5692973444832684,
 0.7554052155146554,
 0.47268560826224704,
 0.40618976231888965,
 0.40418818603445095,
 0.4901150873760632,
 0.8220635827003541,
 0.813926508329709,
 0.44397169818512455,
 0.34249940781038785,
 0.2771340681822595,
 0.2857232739995457,
 0.416362539377805,
 0.42139346709240605,
 -0.4607039310380989,
 0.35063669905760736,
 0.29025943379380287,
 0.20161767642435285,
 0.14899449669391363,
 0.1391050892048101,
 0.101575470368081,
 -0.43268781976857773,
 -0.7377673147129719,
 0.2811978313035723,
 0.2412668521070741,
 0.16143624390285072,
 0.07687342847749522,
 -0.004886370668719575,
 -0.12716322843761613,
 -0.4577482944504797,
 -0.6990282431762781,
 -0.949915822935513,
 0.24237943146502194,
 0.201871692

In [86]:
# FOAMFileを作成し、読み込ませる。
def makeFoamFile(time_step,TU_Parsed):
    path_pmv = './{}/PMV'.format(time_step)
    path_ppd = './{}/PPD'.format(time_step)
    def header(time_step,filename):
        header = """/*--------------------------------*- C++ -*----------------------------------*\
=========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     | Website:  https://openfoam.org
    \\  /    A nd           | Version:  6
     \\/     M anipulation  |
\*---------------------------------------------------------------------------*/
FoamFile
{{
    version     2.0;
    format      ascii;
    class       volScalarField;
    location    "{}";
    object      {};
}}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //
""".format(time_step, filename)
        return header
    
    demensions = """
dimensions      [0 0 0 0 0 0 0];

internalField   nonuniform List<scalar> 
{}
(
""".format(nCells)
        
    boundary="""
)
;

boundaryField
{
    ".*"
    {
        type            zeroGradient;
    }
}


// ************************************************************************* //
"""
    str_= np.frompyfunc(str,1,1)
    PMV = '\n'.join(str_(P[0]))
    PPD = '\n'.join(str_(P[1]))
    
    f = open(path_pmv, 'w') # ファイルを開く(該当ファイルがなければ新規作成)
    g = open(path_ppd, 'w')
    f.write(header(time_step,"PMV")) # 文字列を記載する
    g.write(header(time_step,"PPD"))
    f.write(demensions)
    g.write(demensions)
    f.write(PMV)
    g.write(PPD)
    f.write(boundary)
    g.write(boundary)
    
    f.close() 
    g.close() 

In [87]:
makeFoamFile(100,778)

In [ ]:
time_step=0
nCells=778
path_pmv = './{}/PMV'.format(time_step)
path_ppd = './{}/PPD'.format(time_step)
def header(time_step,filename):
    header = """/*--------------------------------*- C++ -*----------------------------------*\
=========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     | Website:  https://openfoam.org
    \\  /    A nd           | Version:  6
     \\/     M anipulation  |
\*---------------------------------------------------------------------------*/
FoamFile
{{
    version     2.0;
    format      ascii;
    class       volScalarField;
    location    "{}";
    object      {};
}}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //
""".format(time_step, filename)
    return header

demensions = """
dimensions      [0 0 0 0 0 0 0];

internalField uniform {};
""".format()
    
boundary="""
)
;

boundaryField
{
    ".*"
    {
        type            zeroGradient;
    }
}


// ************************************************************************* //
"""
str_= np.frompyfunc(str,1,1)
PMV = '\n'.join(str_(P[0]))
PPD = '\n'.join(str_(P[1]))

f = open(path_pmv, 'w') # ファイルを開く(該当ファイルがなければ新規作成)
g = open(path_ppd, 'w')
f.write(header(time_step,"PMV")) # 文字列を記載する
g.write(header(time_step,"PPD"))
f.write(demensions)
g.write(demensions)
f.write(PMV)
g.write(PPD)
f.write(boundary)
g.write(boundary)

f.close() 
g.close() 

In [80]:
PMV

'0.6180338641846064\n0.3264824877239651\n0.548682542796134\n0.29501308532441783\n0.4146860037250739\n0.546941242028897\n0.3306797251840607\n0.47175703876910774\n0.4820305260001097\n0.6146573378595112\n0.3887718370724176\n0.47824370934352434\n0.7058237629714709\n0.5692973444832684\n0.7554052155146554\n0.47268560826224704\n0.40618976231888965\n0.40418818603445095\n0.4901150873760632\n0.8220635827003541\n0.813926508329709\n0.44397169818512455\n0.34249940781038785\n0.2771340681822595\n0.2857232739995457\n0.416362539377805\n0.42139346709240605\n-0.4607039310380989\n0.35063669905760736\n0.29025943379380287\n0.20161767642435285\n0.14899449669391363\n0.1391050892048101\n0.101575470368081\n-0.43268781976857773\n-0.7377673147129719\n0.2811978313035723\n0.2412668521070741\n0.16143624390285072\n0.07687342847749522\n-0.004886370668719575\n-0.12716322843761613\n-0.4577482944504797\n-0.6990282431762781\n-0.949915822935513\n0.24237943146502194\n0.20187169283494802\n0.13942397951861546\n0.0531753467826

In [100]:
T_test = ParsedParameterFile(CASE.name + '/' + '50' + '/T')
U_test = ParsedParameterFile(CASE.name + '/' + '50' + '/U')

In [99]:
T = np.array(T_test['internalField'])
U = np

'nonuniform List<scalar> 775
(
  300.063
  300.021
  300.015
  300.012
  300.01
  300.009
  300.009
  300.008
  300.008
  300.008
  300.008
  300.008
  300.01
  300.013
  300.02
  300.044
  300.018
  300.01
  300.007
  300.006
  300.005
  300.005
  300.006
  300.006
  300.008
  300.01
  300.016
  300.032
  300.065
  300.052
  300.056
  300.065
  300.12
  300.071
  300.021
  300.01
  300.006
  300.003
  300.002
  300.001
  300.001
  300.001
  300
  300
  300.001
  300.001
  300.002
  300.004
  300.009
  300.016
  300.03
  300.023
  300.008
  300.001
  299.998
  299.997
  299.996
  299.996
  299.997
  299.997
  299.999
  300.001
  300.007
  300.025
  300.056
  300.048
  300.055
  300.07
  300.134
  300.081
  300.024
  300.01
  300.005
  300.002
  300.001
  300
  300
  299.999
  299.999
  299.999
  299.999
  299.999
  299.999
  299.999
  299.999
  300.001
  300.005
  300.003
  299.998
  299.996
  299.995
  299.994
  299.994
  299.994
  299.994
  299.995
  299.996
  299.999
  300.004
  300

In [19]:
a.change_control(3)

In [17]:
memory = pd.DataFrame(columns=['control','step','runOK','time'])
for i in range(1,5):
    a.change_control(i)
    a.reset()
    for j in range(1,11):
        start = time.time()
        _,_,runOK = a.step(np.random.randint(0,26))
        elapsed_time = time.time() - start
        series = pd.Series([i,j,runOK,elapsed_time],index=memory.columns)
        memory = memory.append(series, ignore_index = True)
    

In [18]:
memory

,control,step,runOK,time
0,1,1,True,1.860405
1,1,2,True,1.779847
2,1,3,True,1.799400
3,1,4,True,1.823438
4,1,5,True,1.757091
5,1,6,True,1.804210
6,1,7,True,1.802775
7,1,8,True,1.770538
8,1,9,True,2.153744
9,1,10,True,2.352690


In [178]:
#action = agent.get_action(observation, episode)
observation_next,reward,done,info = env.step(1)

In [22]:
#def __init__(self):
Env = Aircond(CASE)  # 実行する課題を設定
num_states = Env.observation_space.shape[0]*Env.observation_space.shape[1]  # 課題の状態数を取得
num_actions = Env.action_space.shape[0]  # 行動を取得
agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

Sequential(
  (fc1): Linear(in_features=597, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=27, bias=True)
)


In [25]:
env =gym.make(ENV)

In [33]:
    observation = Env.reset()  # 環境の初期化

    state = observation  # 観測をそのまま状態sとして使用
    state = torch.from_numpy(state).type(
        torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
    #state = torch.unsqueeze(state,0)  # size 4をsize 1x4に変換

In [15]:

#def run(self):
#    '''実行'''
episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
complete_episodes = 0  # 195step以上連続で立ち続けた試行数
episode_final = False  # 最後の試行フラグ
frames = []  # 最後の試行を動画にするために画像を格納する変数

for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
    observation = Env.reset()  # 環境の初期化

    state = observation  # 観測をそのまま状態sとして使用
    state = torch.from_numpy(state).type(
        torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
    state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

    for step in range(MAX_STEPS):  # 1エピソードのループ

        if episode_final is True:  # 最終試行ではframesに各時刻の画像を追加していく
            frames.append(self.env.render(mode='rgb_array'))

        action = self.agent.get_action(state, episode)  # 行動を求める
        
        # 行動a_tの実行により、s_{t+1}とdoneフラグを求める
        # actionから.item()を指定して、中身を取り出す
        observation_next, _, done, _ = self.env.step(
            action.item())  # rewardとinfoは使わないので_にする

        # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
        if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
            state_next = None  # 次の状態はないので、Noneを格納

            # 直近10episodeの立てたstep数リストに追加
            episode_10_list = np.hstack(
                (episode_10_list[1:], step + 1))

            if step < 195:
                reward = torch.FloatTensor(
                    [-1.0])  # 途中でこけたら罰則として報酬-1を与える
                complete_episodes = 0  # 連続成功記録をリセット
            else:
                reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
                complete_episodes = complete_episodes + 1  # 連続記録を更新
        else:
            reward = torch.FloatTensor([0.0])  # 普段は報酬0
            state_next = observation_next  # 観測をそのまま状態とする
            state_next = torch.from_numpy(state_next).type(
                torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
            state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

        # メモリに経験を追加
        self.agent.memorize(state, action, state_next, reward)

        # Experience ReplayでQ関数を更新する
        self.agent.update_q_function()

        # 観測の更新
        state = state_next

        # 終了時の処理
        if done:
            print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                episode, step + 1, episode_10_list.mean()))
            break

    if episode_final is True:
        # 動画を保存と描画
        display_frames_as_gif(frames)
        break

    # 10連続で200step経ち続けたら成功
    if complete_episodes >= 10:
        print('10回連続成功')
        episode_final = True  # 次の試行を描画を行う最終試行とする
    
    
    return action

In [ ]:
import pdb; pdb.set_trace()

In [5]:
# CartPoleを実行する環境のクラスです


class Environment:

    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態数4を取得
        num_actions = self.env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

        
    def run(self):
        '''実行'''
        episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        episode_final = False  # 最後の試行フラグ
        frames = []  # 最後の試行を動画にするために画像を格納する変数

        for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
            observation = self.env.reset()  # 環境の初期化

            state = observation  # 観測をそのまま状態sとして使用
            state = torch.from_numpy(state).type(
                torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
            state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

            for step in range(MAX_STEPS):  # 1エピソードのループ

                if episode_final is True:  # 最終試行ではframesに各時刻の画像を追加していく
                    frames.append(self.env.render(mode='rgb_array'))

                action = self.agent.get_action(state, episode)  # 行動を求める
                
                # 行動a_tの実行により、s_{t+1}とdoneフラグを求める
                # actionから.item()を指定して、中身を取り出す
                observation_next, _, done, _ = self.env.step(
                    action.item())  # rewardとinfoは使わないので_にする

                # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
                if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
                    state_next = None  # 次の状態はないので、Noneを格納

                    # 直近10episodeの立てたstep数リストに追加
                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))

                    if step < 195:
                        reward = torch.FloatTensor(
                            [-1.0])  # 途中でこけたら罰則として報酬-1を与える
                        complete_episodes = 0  # 連続成功記録をリセット
                    else:
                        reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
                        complete_episodes = complete_episodes + 1  # 連続記録を更新
                else:
                    reward = torch.FloatTensor([0.0])  # 普段は報酬0
                    state_next = observation_next  # 観測をそのまま状態とする
                    state_next = torch.from_numpy(state_next).type(
                        torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
                    state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

                # メモリに経験を追加
                self.agent.memorize(state, action, state_next, reward)

                # Experience ReplayでQ関数を更新する
                self.agent.update_q_function()

                # 観測の更新
                state = state_next

                # 終了時の処理
                if done:
                    print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break
            if episode_final is True:
                # 動画を保存と描画
                display_frames_as_gif(frames)
                break

            # 10連続で200step経ち続けたら成功
            if complete_episodes >= 10:
                print('10回連続成功')
                episode_final = True  # 次の試行を描画を行う最終試行とする
                
            
            return action

In [17]:
# main クラス
cartpole_env = Environment()
cartpole_env.run()


Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0 Episode: Finished after 16 steps：10試行の平均step数 = 1.6
> <ipython-input-16-40644ceb0f2e>(88)run()
-> return action
(Pdb) p observation
array([-0.03617627,  0.02394902, -0.02056083,  0.03398385])
(Pdb) p state
None
(Pdb) p action
tensor([[0]])
--KeyboardInterrupt--
(Pdb) aa
*** NameError: name 'aa' is not defined
(Pdb) exit


BdbQuit: 